<a href="https://colab.research.google.com/github/raz0208/ModernBERT/blob/main/ModernBERT_TokenEmbedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Extract embedding form inpot text using ModernBERT

In [ ]:
# !pip install transformers spacy keybert

In [ ]:
# import required libraries
import os
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import spacy
from keybert import KeyBERT
import re

### Load NLP and ModernBert models

In [ ]:
# Load NLP model for noun phrase extraction
nlp = spacy.load("en_core_web_sm")

# Load ModernBERT tokenizer and model from Hugging Face
MODEL_NAME = "answerdotai/ModernBERT-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

### Corpus (text) cleaning

In [ ]:
# Function to clean text
def clean_text_for_topics(text):
    # Remove content in parentheses (e.g., (2015), (note))
    text = re.sub(r'\([^)]*\)', '', text)

    # Remove percentages, currency, units like "mg", "billion", "years", etc.
    text = re.sub(r'\b\d+(\.\d+)?\s*(%|percent|million|billion|thousand|mg|kg|g|km|cm|years?|months?|days?)\b', '', text, flags=re.IGNORECASE)

    # Remove standalone numbers (integers, decimals, years)
    text = re.sub(r'\b\d+(\.\d+)?\b', '', text)

    # Remove numeric ranges (e.g., 2020–2023 or 10-15)
    text = re.sub(r'\b\d{2,4}\s*[-–—]\s*\d{2,4}\b', '', text)

    # Remove metadata indicators like "Deaths:", "Rate:", "Total:"
    text = re.sub(r'\b(Deaths?|Rate|Cases?|Total|Incidence|Prevalence|Statistics):?.*', '', text, flags=re.IGNORECASE)

    # Remove slashes and colons separating stats or metadata
    text = re.sub(r'[:/]', ' ', text)

    # Remove all punctuation except commas
    text = re.sub(r'[^\w\s,]', '', text)

    # Collapse extra spaces
    text = re.sub(r'\s{2,}', ' ', text)

    return text.strip()

### Extract emmbedings based on full text and each token separatly.

In [ ]:
# Function to get text and return the embedding
def get_text_embedding(text):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # Forward pass to get hidden states
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the embeddings (use CLS token for sentence-level embedding)
    cls_embedding = outputs.last_hidden_state[:, 0, :]  # shape: [batch_size, hidden_size]

    return cls_embedding.squeeze().numpy()

In [ ]:
# Function to get per-token embeddings
def get_token_embeddings(text):

    # Call text cleaning function
    text = clean_text_for_topics(text)

    # Tokenize with return_tensors and also get token strings
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, return_attention_mask=True)
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

    # Forward pass
    with torch.no_grad():
        outputs = model(**inputs)

    embeddings = outputs.last_hidden_state.squeeze(0)  # shape: [seq_len, hidden_size]

    # Prepare cleaned token list and their embeddings
    token_embeddings = []
    for token, emb in zip(tokens, embeddings):
        # Skip special tokens and punctuation/whitespace
        if token in tokenizer.all_special_tokens:
            continue
        token = token.lstrip("Ġ▁")  # Remove leading space markers from BPE/SentencePiece
        if not token.isalnum():  # Skip non-alphanumeric tokens
            continue
        token_embeddings.append((token, emb.numpy()))

    return token_embeddings

### Topic extraction: First approch

In [ ]:
# ### --- ### Basic method for topic sxtraction ### --- ###

# # Extract topics (noun phrase) from sentence
# def extract_topics(text):
#     doc = nlp(text)
#     topics = []

#     for chunk in doc.noun_chunks:
#         # Remove articles/determiners (like "the", "an", "a")
#         words = [token.text for token in chunk if token.pos_ != "DET"]
#         if words:
#             cleaned_phrase = " ".join(words).strip()
#             topics.append(cleaned_phrase)

#     # Remove duplicates while preserving order
#     seen = set()
#     unique_topics = []
#     for topic in topics:
#         if topic.lower() not in seen:
#             unique_topics.append(topic)
#             seen.add(topic.lower())

#     return unique_topics

In [ ]:
# ### --- ### Second method for topic sxtraction ### --- ###

# # Extract topics (noun phrase) from sentence
# def extract_topics(text):
#     doc = nlp(text)
#     topics = []

#     # 1. Add noun chunks (excluding determiners)
#     for chunk in doc.noun_chunks:
#         words = [token.text for token in chunk if token.pos_ != "DET"]
#         if words:
#             cleaned = " ".join(words).strip()
#             topics.append(cleaned)

#     # 2. Add standalone noun tokens (important for medical terms like "diabetes")
#     for token in doc:
#         if token.pos_ == "NOUN" and not token.is_stop:
#             topics.append(token.text.strip())

#     # 3. Add named entities related to medical types
#     for ent in doc.ents:
#         if ent.label_ in {"DISEASE", "CONDITION", "MEDICAL_CONDITION", "HEALTH_CONDITION", "SYMPTOM"}:
#             topics.append(ent.text.strip())

#     # 4. Clean and deduplicate
#     seen = set()
#     unique_topics = []
#     for topic in topics:
#         topic_clean = topic.lower()
#         if topic_clean not in seen and topic_clean.isalpha():
#             unique_topics.append(topic)
#             seen.add(topic_clean)

#     return unique_topics

In [ ]:
### --- ### Third method for topic sxtraction ### --- ###

# Extract topics (noun phrase) from sentence
def extract_topics(text):
    doc = nlp(text)
    topics = []

    for chunk in doc.noun_chunks:
        # Keep only noun-based chunks
        if all(token.pos_ in ["NOUN", "PROPN", "ADJ"] for token in chunk if not token.is_stop):
            topic = " ".join(token.text for token in chunk if token.pos_ != "DET").strip()
            topics.append(topic)

    # Fallback: add standalone nouns that weren't in noun_chunks
    for token in doc:
        if token.pos_ in ["NOUN", "PROPN"] and not token.is_stop:
            if not any(token.text in topic for topic in topics):
                topics.append(token.text)

    # Remove duplicates
    seen = set()
    unique_topics = []
    for topic in topics:
        if topic.lower() not in seen:
            unique_topics.append(topic)
            seen.add(topic.lower())

    return unique_topics

## Topic Extraction: Second approch
- Extract the representative topic based on the input text.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Function to extract representative topic
def extract_main_topic(text):
    """
    Extract the most representative topic from the input text using ModernBERT embeddings.
    """
    # Step 1: Extract all topics from text
    candidate_topics = extract_topics(text)

    if not candidate_topics:
        return None, None

    # Step 2: Get embedding for full input text
    text_embedding = get_text_embedding(text).reshape(1, -1)

    # Step 3: Compute embeddings and similarities for each candidate topic
    best_topic = None
    highest_similarity = -1

    for topic in candidate_topics:
        topic_embedding = get_text_embedding(topic).reshape(1, -1)
        similarity = cosine_similarity(text_embedding, topic_embedding)[0][0]

        if similarity > highest_similarity:
            highest_similarity = similarity
            best_topic = topic

    return best_topic, highest_similarity

### Run the code

In [ ]:
### --- ### Sample text for test ### --- ###

# 1- This is an application about Breast Cancer.
# 2- Treating high blood pressure, high blood lipids, diabetes.
# 3- Heart failure, heart attack, stroke, aneurysm, peripheral artery disease, sudden cardiac arrest. Deaths: 17.9 million / 32% (2015)

In [ ]:
# Example usage (Sentence: This is an application about Breast Cancer.)
if __name__ == "__main__":
    user_text = input("Enter your text: ")

    # Get sentence embedding
    embedding = get_text_embedding(user_text)
    print("\nSentence Embedding vector shape:", embedding.shape)
    print("Sentence Embedding (first 10 values):", embedding[:10])

    print("\n", "####"*10)

    # Get per-token embeddings
    token_embeddings = get_token_embeddings(user_text)
    print("\nToken-wise Embeddings:")
    for token, emb in token_embeddings:
        print(f"Token: {token:15} | Embedding (first 5 vals): {emb[:5]}")

    print("\n", "####"*10)

    # Extract topic and embedding
    topics = extract_topics(user_text)
    if topics:
        print(f"\nIdentified topics: {topics}")
        for topic in topics:
            topic_embedding = get_text_embedding(topic)
            print(f"\nTopic: '{topic}'")
            print("Embedding shape:", topic_embedding.shape)
            print("Embedding (first 5 values):", topic_embedding[:5])
    else:
        print("No topics found.")

    print("\n", "####"*10)

    # Extract main topic
    Representative_topic, similarity = extract_main_topic(user_text)
    if Representative_topic:
        print(f"\nRepresentative Topic: '{Representative_topic}' (similarity: {similarity:.4f})")

        # Get sentence embedding
        RepTopic_embedding = get_text_embedding(Representative_topic)
        print("\nSentence Embedding vector shape:", RepTopic_embedding.shape)
        print("Sentence Embedding (first 10 values):", RepTopic_embedding[:10])
    else:
        print("No main topic found.")

Enter your text: This is an application about Breast Cancer.

Sentence Embedding vector shape: (768,)
Sentence Embedding (first 10 values): [ 0.42236355 -0.8862073  -0.6536694  -0.2981413  -0.5874422  -0.720903
 -0.8588484  -0.89695704  0.5856571  -0.9214181 ]

 ########################################

Token-wise Embeddings:
Token: This            | Embedding (first 5 vals): [-0.82628614 -1.0354643  -0.4054439  -0.11625224  0.6733342 ]
Token: is              | Embedding (first 5 vals): [ 0.70604783 -1.96604    -0.57871556 -0.09674171  0.31713882]
Token: an              | Embedding (first 5 vals): [ 0.99583006 -1.5493056  -0.47588482  0.37580994 -0.02157765]
Token: application     | Embedding (first 5 vals): [ 0.4509778  -2.7144003   0.61317146 -0.36472976  1.0529627 ]
Token: about           | Embedding (first 5 vals): [ 2.005485   -1.0684667  -0.6908237  -0.14003669 -0.41675222]
Token: Breast          | Embedding (first 5 vals): [ 1.3654128  -2.0563452  -1.4072521   0.11204606  0.1139